In [1]:
import pandas as pd
import numpy as np

In [2]:
def analyze_dataframe(df):
    results = []
    for i, col_name in enumerate(df.columns):
        col = df[col_name]
        
        # 获取列数据类型
        dtype = col.dtype
        
        # 统计特定值
        count_neg1 = (col == -1).sum() if dtype != object else (col.astype(str) == '-1').sum()
        count_0 = (col == 0).sum() if dtype != object else (col.astype(str) == '0').sum()
        count_1 = (col == 1).sum() if dtype != object else (col.astype(str) == '1').sum()
        count_2 = (col == 2).sum() if dtype != object else (col.astype(str) == '2').sum()
        
        # 统计正负值（排除0）
        if pd.api.types.is_numeric_dtype(col):
            count_positive = (col > 0).sum()
            count_negative = (col < 0).sum()
        else:
            count_positive = count_negative = 0
        
        # 统计空值
        count_null = col.isna().sum()
        
        # 格式化输出
        result = (
            f"第{i}列，{col_name}，"
            f"该列元素为{dtype}类型，"
            f"有{count_neg1}个值为-1，"
            f"{count_0}个值为0，"
            f"有{count_1}个值为1，"
            f"有{count_2}个值为2，"
            f"有{count_positive}个正数值，"
            f"{count_negative}个负数值，"
            f"{count_null}个值为空值"
        )
        results.append(result)
    
    return results

In [3]:
data_batch = "train"
data_target = "rent"
fankui = ""
path = f"结构化数据{fankui}_{data_batch}_{data_target}.csv"
df = pd.read_csv(path)

In [4]:
df = df.drop("客户反馈", axis=1)

In [5]:
# 分析并打印结果
analysis_results = analyze_dataframe(df)
for result in analysis_results:
    print(result)

第0列，城市，该列元素为int64类型，有0个值为-1，14542个值为0，有7725个值为1，有12059个值为2，有84357个正数值，0个负数值，0个值为空值
第1列，Price，该列元素为float64类型，有0个值为-1，0个值为0，有0个值为1，有0个值为2，有98899个正数值，0个负数值，0个值为空值
第2列，lon，该列元素为float64类型，有0个值为-1，0个值为0，有0个值为1，有0个值为2，有98899个正数值，0个负数值，0个值为空值
第3列，lat，该列元素为float64类型，有0个值为-1，0个值为0，有0个值为1，有0个值为2，有98899个正数值，0个负数值，0个值为空值
第4列，年份，该列元素为float64类型，有0个值为-1，0个值为0，有0个值为1，有0个值为2，有98899个正数值，0个负数值，0个值为空值
第5列，区县，该列元素为float64类型，有0个值为-1，0个值为0，有0个值为1，有0个值为2，有94222个正数值，0个负数值，4677个值为空值
第6列，板块，该列元素为float64类型，有0个值为-1，0个值为0，有43个值为1，有23个值为2，有93755个正数值，0个负数值，5144个值为空值
第7列，coord_x，该列元素为float64类型，有0个值为-1，0个值为0，有0个值为1，有0个值为2，有94268个正数值，0个负数值，4631个值为空值
第8列，coord_y，该列元素为float64类型，有0个值为-1，0个值为0，有0个值为1，有0个值为2，有94268个正数值，0个负数值，4631个值为空值
第9列，室数，该列元素为int64类型，有1个值为-1，5094个值为0，有23102个值为1，有34047个值为2，有93804个正数值，1个负数值，0个值为空值
第10列，厅数，该列元素为int64类型，有1个值为-1，11007个值为0，有48748个值为1，有38781个值为2，有87891个正数值，1个负数值，0个值为空值
第11列，厨房数，该列元素为int64类型，有1个值为-1，98898个值为0，有0个值为1，有0个值为2，有0个正数值，1个负数值，0个值为空值
第12列，卫数，该列元素为int64类型，有1个值为-1，64482个值为0，有25717个值为1，有793

In [6]:
def process_dataframe(df):
    """
    处理DataFrame的每一列：
    1. 如果-1占比超过20%，删除该列
    2. 如果-1占比小于5%，用中位数填充相应值
    """
    # 创建副本避免修改原始数据
    processed_df = df.copy()
    # 记录要删除的列
    cols_to_drop = []
    
    for col in processed_df.columns:
        # 只处理数值型列
#         if col in fix_column:
#             continue
        if pd.api.types.is_numeric_dtype(processed_df[col]):
            col_data = processed_df[col]
            n = len(col_data)
            
            # 计算-1的占比
            minus_one_count = (col_data == -1).sum()
            minus_one_ratio = minus_one_count / n
            
            # 情况1：-1占比超过20%
            if minus_one_ratio > 0.2:
                cols_to_drop.append(col)
                print(f"已删去{col}列（-1占比{minus_one_ratio:.2%}）")
                continue
            
            # 计算0的占比
            zero_count = (col_data == 0).sum()
            zero_ratio = zero_count / n
            
            # 计算-1的总占比
            invalid_ratio = (minus_one_count) / n
            
            # 情况2：-1中位数填充
            if invalid_ratio > 0 and invalid_ratio < 0.05:
                # 确定哪些值需要填充
                to_fill = []
                if minus_one_count > 0:
                    to_fill.append("-1")
                
                # 计算有效值的中位数
                valid_values = col_data[(col_data != -1)]
                if not valid_values.empty:
                    median_val = valid_values.median()
                    
                    # 填充-1和0的位置
                    mask = (col_data == -1)
                    processed_df.loc[mask, col] = median_val
                    
                    print(f"{col}列已对-1进行中位数填充（{minus_one_count}个-1中位数={median_val:.4f}）")
            
    
    # 删除需要删除的列
    processed_df.drop(columns=cols_to_drop, inplace=True)
    return processed_df

In [7]:
processed_df = process_dataframe(df)

室数列已对-1进行中位数填充（1个-1中位数=2.0000）
厅数列已对-1进行中位数填充（1个-1中位数=1.0000）
厨房数列已对-1进行中位数填充（1个-1中位数=0.0000）
卫数列已对-1进行中位数填充（1个-1中位数=0.0000）
是否有电梯列已对-1进行中位数填充（4个-1中位数=1.0000）
是否有燃气列已对-1进行中位数填充（4582个-1中位数=1.0000）
已删去租赁时间列（-1占比47.46%）
已删去容积率列（-1占比24.35%）
已删去物业费列（-1占比22.41%）
已删去燃气费数值列（-1占比25.34%）
已删去供热费数值列（-1占比70.84%）
已删去停车位数量列（-1占比25.76%）
已删去停车费列（-1占比35.07%）


In [8]:
def fill_zero_with_median(df, column_name):
    """
    将指定列中的0值替换为该列除0以外元素的中位数
    
    参数:
    df -- 要处理的DataFrame
    column_name -- 要处理的列名
    
    返回:
    处理后的DataFrame副本
    """
    # 创建DataFrame副本以避免修改原始数据
    processed_df = df.copy()
    
    # 检查列是否存在
    if column_name not in processed_df.columns:
        raise ValueError(f"列 '{column_name}' 不存在于DataFrame中")
    # 获取指定列的数据
    col_data = processed_df[column_name]
    
    
    # 计算0的数量
    zero_count = (col_data == 0).sum()
    
    if zero_count == 0:
        print(f"列 '{column_name}' 中没有0值，无需处理")
        return processed_df
    # 计算非零元素的中位数
    non_zero_values = col_data[col_data != 0]
    median_val = non_zero_values.median()
    # 替换0值为中位数
    processed_df.loc[col_data == 0, column_name] = median_val
    
    print(f"列 '{column_name}' 已处理: {zero_count}个0值被替换为中位数 {median_val:.4f}")
    return processed_df

In [9]:
proprocessed_df = fill_zero_with_median(processed_df, '总楼层')

列 '总楼层' 已处理: 69个0值被替换为中位数 18.0000


In [11]:
proprocessed_df.to_csv(f"正常化数据{fankui}_{data_batch}_{data_target}.csv", index=False)